# setup

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
import numpy as np
import os
import glob
import cv2 as cv

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'		  ml_reciclaje
 confusion_matrix.ipynb		 'My Drive'
 deep_learning			  normalImages.zip
 dummy_augmented		 'resultados modelo yolov4.gsheet'
 images_to_train_with_augmented   Tesis
 IMG_20220911_213738.jpg	  yolov3_augmented
 IMG_20220911_213826.jpg	  yolov3_with_dummies_full_augmented
 IMG_20220911_213853.jpg	  yolov4_augmented
 ITAM				  yolov4_images_to_train_plus_two_folders_dummy
 map.gsheet			  yolov4_with_dummies_full_augmented


# Unzip images and load them

In [3]:
%cd /content/
!unzip /mydrive/deep_learning/preprocessed.zip
!unzip /mydrive/deep_learning/gaussian_images.zip

Streaming output truncated to the last 5000 lines.
  inflating: gaussian_images/train/4252_train_blured.png  
  inflating: gaussian_images/train/10267_train_blured.png  
  inflating: gaussian_images/train/18589_train_blured.png  
  inflating: gaussian_images/train/10259_train_blured.png  
  inflating: gaussian_images/train/13413_train_blured.png  
  inflating: gaussian_images/train/7426_train_blured.png  
  inflating: gaussian_images/train/15011_train_blured.png  
  inflating: gaussian_images/train/1024_train_blured.png  
  inflating: gaussian_images/train/21576_train_blured.png  
  inflating: gaussian_images/train/8855_train_blured.png  
  inflating: gaussian_images/train/19815_train_blured.png  
  inflating: gaussian_images/train/24503_train_blured.png  
  inflating: gaussian_images/train/4051_train_blured.png  
  inflating: gaussian_images/train/521_train_blured.png  
  inflating: gaussian_images/train/10064_train_blured.png  
  inflating: gaussian_images/train/2453_train_blured.png

In [4]:
!rm -r __MACOSX

In [6]:
!mkdir data

In [7]:
!mv preprocessed/ data/
!mv gaussian_images/ data/

In [9]:
!cp /mydrive/deep_learning/preprocessed_train.txt data/
!cp /mydrive/deep_learning/preprocessed_valid.txt data/
!cp /mydrive/deep_learning/gaussian_train.txt data/
!cp /mydrive/deep_learning/gaussian_valid.txt data/

In [21]:
base_path = os.path.join("../", "data")

In [22]:
x_train_path = os.path.join(base_path, "preprocessed", "train")
x_valid_path = os.path.join(base_path, "preprocessed", "valid")

x_train_gaussian_path = os.path.join(base_path, "gaussian_images", "train")
x_valid_gaussian_path = os.path.join(base_path, "gaussian_images", "valid")

In [23]:
print(x_train_path)
print(x_valid_path)
print(x_train_gaussian_path)
print(x_valid_gaussian_path)

../data/preprocessed/train
../data/preprocessed/valid
../data/gaussian_images/train
../data/gaussian_images/valid


# get image's paths: x_train and x_valid

In [24]:
x_train_file = os.path.join(base_path, "preprocessed_train.txt")
with open(x_train_file) as ptf:
  x_train = ptf.read().splitlines()
ptf.close()

In [25]:
x_train[0]

'preprocessed/train/10000_train.png'

In [26]:
x_valid_file = os.path.join(base_path, "preprocessed_valid.txt")
with open(x_valid_file) as pvf:
  x_valid = pvf.read().splitlines()
pvf.close()

In [27]:
x_valid[0]

'preprocessed/valid/1000_valid.png'

## get image's paths: x_train_gaussian and x_valid_gaussian

In [28]:
x_train_gaussian_file = os.path.join(base_path, "gaussian_train.txt")
with open(x_train_gaussian_file) as xgtf:
  x_gaussian_train = xgtf.read().splitlines()
xgtf.close()

In [29]:
x_gaussian_train[0]

'gaussian_images/train/10000_train_blured.png'

In [30]:
x_valid_gaussian_file = os.path.join(base_path, "gaussian_valid.txt")
with open(x_valid_gaussian_file) as xgvf:
  x_gaussian_valid = xgvf.read().splitlines()
xgvf.close()

In [31]:
x_gaussian_valid[0]

'gaussian_images/valid/1000_valid_blured.png'

In [32]:
print("Dimensiones para imagenes originales:", len(x_train), len(x_valid))
print("Dimensiones para imagenes gaussianas:", len(x_gaussian_train), len(x_gaussian_valid))

Dimensiones para imagenes originales: 25462 2121
Dimensiones para imagenes gaussianas: 25462 2121


In [33]:
x_train = x_train[:15000]
x_valid = x_valid[:500]
x_gaussian_train = x_gaussian_train[:15000]
x_gaussian_valid = x_gaussian_valid[:500]

# Autoencoder

In [52]:
OW, NW = 52, 52
OH, NH = 64, 64

In [54]:
# Crear las listas para las imágenes originales y con ruido
original_images = []
noisy_images = []

# Leer las imágenes originales y con ruido
for or_img, ns_img in zip(x_train, x_gaussian_train):

    # Leer las imágenes en escala de grises
    original_image = cv.imread(os.path.join(base_path, or_img))
    noisy_image = cv.imread(os.path.join(base_path, ns_img))

    oh, ow, _ = original_image.shape
    nh, nw, _ = noisy_image.shape


    original_image = cv.resize(original_image, (OW, OH), interpolation = cv.INTER_AREA)
    noisy_image = cv.resize(noisy_image, (NW, NH), interpolation = cv.INTER_AREA)

    # Normalizar los valores de píxeles en el rango [0, 1]
    original_image = original_image.astype(np.float32) / 255.0
    noisy_image = noisy_image.astype(np.float32) / 255.0

    # Agregar las imágenes a las listas
    original_images.append(original_image)
    noisy_images.append(noisy_image)

# Asegurarse de que las imágenes tengan la forma adecuada
original_images = np.expand_dims(original_images, axis=-1)
noisy_images = np.expand_dims(noisy_images, axis=-1)

In [55]:
# Definir la arquitectura del autoencoder

# Definir la arquitectura del autoencoder
def build_autoencoder():
    # Encoder
    input_img = tf.keras.Input(shape=(64, 52, 3))
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    decoded = tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Autoencoder model
    autoencoder = tf.keras.Model(input_img, decoded)
    return autoencoder

# Construir y compilar el modelo del autoencoder
autoencoder = build_autoencoder()
autoencoder.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 64, 52, 3)]       0         
                                                                 
 conv2d_70 (Conv2D)          (None, 64, 52, 32)        896       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 32, 26, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_71 (Conv2D)          (None, 32, 26, 64)        18496     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 16, 13, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_72 (Conv2D)          (None, 16, 13, 64)        369

In [56]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Cargar los datos de entrenamiento (ejemplo)
# Aquí debes reemplazar 'X_train' con tus propios datos de entrenamiento
X_train = original_images
Y_train = noisy_images

# Entrenar el autoencoder
autoencoder.fit(Y_train, X_train, epochs=10, batch_size=16)

# Realizar predicciones en nuevos datos
# Aquí debes reemplazar 'X_test' con tus propios datos de prueba
#X_test = np.random.rand(10, 128, 100, 3)
#predictions = autoencoder.predict(X_test)

Epoch 1/10
 77/938 [=>............................] - ETA: 46s - loss: 0.0250

KeyboardInterrupt: 